In [110]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Model
import datetime

In [112]:
train_datagen = image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/faces/target_ariel/',
        target_size=(250,250),
        batch_size=10,
        class_mode='binary',
        classes=['positive', 'negative'])

Found 1054 images belonging to 2 classes.


In [113]:
train_generator.nb_sample

1054

In [50]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(250,250,3))

In [51]:
# add a global spatial average pooling layer
x = base_model.output
#x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x)
x = Flatten()(x)
predictions = Dense(1, activation='softmax')(x)

In [56]:
# this is the model we will train
model = Model(base_model.input, predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(train_generator, 100, 1)

Epoch 1/1
128/100 [======================================] - 3s - loss: 1.2455 - acc: 0.9219


/usr/local/lib/python3.5/dist-packages/Keras-1.2.2-py3.5.egg/keras/engine/training.py:1575: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


In [111]:
train_generator.s

1054

In [89]:
flat = Flatten()(base_model.outputs)

## NOT FAST ENOUGH!!!

In [117]:
train_generator2 = train_datagen.flow_from_directory(
        '/home/ubuntu/faces/target_ariel/',
        target_size=(250,250),
        batch_size=1054,
        class_mode='binary',
        classes=['positive', 'negative'])

Found 1054 images belonging to 2 classes.


In [69]:
from sklearn.linear_model import LogisticRegression

In [70]:
lrm = LogisticRegression()

In [103]:
def fit_transfer(X, y):
    n = X.shape[0]
    X2 = base_model.predict(X)
    X2 = X2.reshape(n, -1)
    lrm.fit(X2, y)

In [118]:
base_model.predict(train_generator2.next()[0]).shape

(1054, 6, 6, 2048)

In [108]:
t = time.time.
base_model.predict(train_generator.next()[0]).reshape(32, -1)

array([[-0.43034479, -0.37844482, -0.72880501, ..., -0.16409096,
        -0.19699332,  0.86325788],
       [-0.43034479, -0.37844482, -0.72880501, ..., -0.16409096,
        -0.19699332,  0.88760197],
       [-0.43034479, -0.37844482, -0.72880501, ..., -0.16409096,
        -0.19699332, -0.30855402],
       ..., 
       [-0.43034479, -0.37844482, -0.72880501, ...,  0.979478  ,
        -0.19699332,  1.7297157 ],
       [-0.43034479,  3.94752526,  4.31892109, ..., -0.16409096,
        -0.19699332,  1.3628751 ],
       [-0.43034479, -0.37844482, -0.72880501, ...,  0.24920583,
        -0.19699332,  0.1857917 ]], dtype=float32)

In [107]:
fit_transfer(train_generator2.next()[0], train_generator2.next()[1])